In [2]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to avoid too many logging messages
import pandas as pd
import numpy as np
import random
import shutil
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
import sklearn
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
#import tensorflow_probability as tfp
#import wandb
#import yaml

from IPython import display as ipd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

from keras_cv_attention_models import efficientnet

In [3]:
class CFG:

    model_name = 'EfficientNetV1B0'

    batch_size = 32
    
    epochs = 5

    folds = 4

    seed = 123

    img_size = [256, 256]

    target_col  = [ "bowel_injury", "extravasation_injury", "kidney_healthy", "kidney_low",
                   "kidney_high", "liver_healthy", "liver_low", "liver_high",
                   "spleen_healthy", "spleen_low", "spleen_high"]

In [4]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    tf.random.set_seed(SEED)
    print('seeding done!!!')

seeding(CFG.seed)

seeding done!!!


In [5]:
df = pd.read_csv('./Dataset/train.csv')
test_df = pd.read_csv('./Dataset/test.csv')

In [6]:
df

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,...,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,injury_name,image_path,width,height
0,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,362,Active_Extravasation,./Dataset/train_images/10004/21057/362.png,512,512
1,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,363,Active_Extravasation,./Dataset/train_images/10004/21057/363.png,512,512
2,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,364,Active_Extravasation,./Dataset/train_images/10004/21057/364.png,512,512
3,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,365,Active_Extravasation,./Dataset/train_images/10004/21057/365.png,512,512
4,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,366,Active_Extravasation,./Dataset/train_images/10004/21057/366.png,512,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13010,9632,1,0,0,1,1,0,0,1,0,...,1,0,0,1,3750,155,Active_Extravasation,./Dataset/train_images/9632/3750/155.png,512,512
13011,9632,1,0,0,1,1,0,0,1,0,...,1,0,0,1,3750,156,Active_Extravasation,./Dataset/train_images/9632/3750/156.png,512,512
13012,9632,1,0,0,1,1,0,0,1,0,...,1,0,0,1,3750,157,Active_Extravasation,./Dataset/train_images/9632/3750/157.png,512,512
13013,9632,1,0,0,1,1,0,0,1,0,...,1,0,0,1,3750,158,Active_Extravasation,./Dataset/train_images/9632/3750/158.png,512,512


In [7]:
df['stratify'] = ''

for col in CFG.target_col:
    df['stratify'] += df[col].astype(str)

df = df.reset_index(drop=True)

sgkf = StratifiedGroupKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.seed)

for  fold , (train_idx, val_idx) in enumerate(sgkf.split(df, df['stratify'], df['patient_id'])):
    df.loc[val_idx, 'fold'] = fold

/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/sklearn/model_selection/_split.py:950: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


In [8]:
def build_decoder():


    def decode_image(path):

        file_binary = tf.io.read_file(path)
        image = tf.image.decode_png(file_binary, channels=3, dtype=tf.uint8)
        image = tf.image.resize(image, CFG.img_size, method='bilinear')
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.reshape(image, [*CFG.img_size, 3])

        return image
    

    def decode_label(label):
        
        label = tf.cast(label, tf.float32)
        
        return (label[0:1], label[1:2], label[2:5], label[5:8], label[8:11])
    
    def decode_with_labels(path, label):

        return decode_image(path), decode_label(label)

    return decode_with_labels
    


def build_dataset(paths, labels, batch_size=32, drop_remainder=False):

    slices = (paths, labels)

    ds = tf.data.Dataset.from_tensor_slices(slices)

    ds = ds.map(build_decoder())

    ds = ds.batch(batch_size, drop_remainder=drop_remainder)

    return ds

In [9]:
def build_model(model_name=CFG.model_name, dim=CFG.img_size):


        # backbone
        base = getattr(efficientnet, model_name)(input_shape=(*dim,3), pretrained='imagenet', num_classes=0)

        inp = base.inputs

        x = base.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)

        # necks
        x_bowel = tf.keras.layers.Dense(32, activation='silu')(x)
        x_extra = tf.keras.layers.Dense(32, activation='silu')(x)
        x_liver = tf.keras.layers.Dense(32, activation='silu')(x)
        x_kidney = tf.keras.layers.Dense(32, activation='silu')(x)
        x_spleen = tf.keras.layers.Dense(32, activation='silu')(x)

        # heads
        out_bowel = tf.keras.layers.Dense(1, name='bowel', activation='sigmoid')(x_bowel)
        out_extra = tf.keras.layers.Dense(1, name='extra', activation='sigmoid')(x_extra)
        out_liver = tf.keras.layers.Dense(3, name='liver', activation='softmax')(x_liver)
        out_kidney = tf.keras.layers.Dense(3, name='kidney', activation='softmax')(x_kidney)
        out_spleen = tf.keras.layers.Dense(3, name='spleen', activation='softmax')(x_spleen)

        out = [out_bowel, out_extra, out_liver, out_kidney, out_spleen]

        model = tf.keras.Model(inputs=inp, outputs=out)

        optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

        loss = {'bowel':tf.keras.losses.BinaryCrossentropy(),
                'extra':tf.keras.losses.BinaryCrossentropy(),
                'liver':tf.keras.losses.CategoricalCrossentropy(),
                'kidney':tf.keras.losses.CategoricalCrossentropy(),
                'spleen':tf.keras.losses.CategoricalCrossentropy()}

        metrics = {'bowel':'accuracy',
                'extra':'accuracy',
                'liver':'accuracy',
                'kidney':'accuracy',
                'spleen':'accuracy'}

        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

        return model


In [10]:

for fold in np.arange(CFG.folds):

    train_df = df.query("fold!=@fold")
    valid_df = df.query("fold==@fold")

    train_paths  = train_df.image_path.values
    train_labels = train_df[CFG.target_col].values.astype(np.float32)
    valid_paths  = valid_df.image_path.values
    valid_labels = valid_df[CFG.target_col].values.astype(np.float32)
    test_paths   = test_df.image_path.values

    K.clear_session()

    model = build_model(CFG.model_name, dim=CFG.img_size)

    train_ds = build_dataset(train_paths, train_labels)
    valid_ds = build_dataset(valid_paths, valid_labels)

    history = model.fit(train_ds, validation_data = valid_ds, epochs=CFG.epochs)
                        
    best_epoch = np.argmin(history.history['val_loss'])
    best_loss = history.history['val_loss'][best_epoch]
    best_acc_bowel = history.history['val_bowel_accuracy'][best_epoch]
    best_acc_extra = history.history['val_extra_accuracy'][best_epoch]
    best_acc_liver = history.history['val_liver_accuracy'][best_epoch]
    best_acc_kidney = history.history['val_kidney_accuracy'][best_epoch]
    best_acc_spleen = history.history['val_spleen_accuracy'][best_epoch]

    best_acc = np.mean([best_acc_bowel, best_acc_extra, best_acc_liver, best_acc_kidney, best_acc_spleen])
    print(f'FOLD {fold} RESULTS')
    print(f'BEST Loss  : {best_loss:.3f}\nBEST Acc   : {best_acc:.3f}\nBEST Epoch : {best_epoch}\n')
    print(f'Bowel : {best_acc_bowel:.3f}')
    print(f'Extravasation : {best_acc_extra:.3f}')
    print(f'Liver: {best_acc_liver:.3f}')
    print(f'Kidney: {best_acc_kidney:.3f}')
    print(f'Spleen: {best_acc_spleen:.3f}')


2023-10-05 19:48:23.832670: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-10-05 19:48:23.832694: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2023-10-05 19:48:23.832698: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2023-10-05 19:48:23.832760: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-05 19:48:23.832795: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


>>>> Load pretrained from: /Users/makoto/.keras/models/efficientnetv1-b0-imagenet.h5
Epoch 1/5


2023-10-05 19:48:35.188787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


299/299 [==============================] - ETA: 0s - loss: 3.3185 - bowel_loss: 0.6980 - extra_loss: 0.5837 - liver_loss: 0.4204 - kidney_loss: 0.6631 - spleen_loss: 0.9533 - bowel_accuracy: 0.5531 - extra_accuracy: 0.7188 - liver_accuracy: 0.8892 - kidney_accuracy: 0.8024 - spleen_accuracy: 0.5969

2023-10-05 19:54:22.098749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

318/318 [==============================] - 506s 1s/step - loss: 3.1525 - bowel_loss: 0.6182 - extra_loss: 0.6338 - liver_loss: 0.4500 - kidney_loss: 0.6302 - spleen_loss: 0.8203 - bowel_accuracy: 0.6850 - extra_accuracy: 0.6545 - liver_accuracy: 0.8663 - kidney_accuracy: 0.8040 - spleen_accuracy: 0.6775 - val_loss: 3.7581 - val_bowel_loss: 0.8770 - val_extra_loss: 0.5527 - val_liver_loss: 0.6788 - val_kidney_loss: 0.6984 - val_spleen_loss: 0.9511 - val_bowel_accuracy: 0.4942 - val_extra_accuracy: 0.7851 - val_liver_accuracy: 0.8346 - val_kidney_accuracy: 0.7529 - val_spleen_accuracy: 0.6200
Epoch 2/5
318/318 [==============================] - 349s 1s/step - loss: 2.6592 - bowel_loss: 0.5415 - extra_loss: 0.5422 - liver_loss: 0.3457 - kidney_loss: 0.4835 - spleen_loss: 0.7462 - bowel_accuracy: 0.7383 - extra_accuracy: 0.7464 - liver_accuracy: 0.9000 - kidney_accuracy: 0.8390 - spleen_accuracy: 0.6949 - val_loss: 3.7989 - val_bowel_loss: 0.8408 - val_extra_loss: 0.6095 - val_liver_loss: 

2023-10-05 21:18:33.802294: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


269/269 [==============================] - ETA: 0s - loss: 3.4256 - bowel_loss: 0.7022 - extra_loss: 0.6416 - liver_loss: 0.6430 - kidney_loss: 0.5690 - spleen_loss: 0.8697 - bowel_accuracy: 0.5477 - extra_accuracy: 0.6705 - liver_accuracy: 0.8161 - kidney_accuracy: 0.8101 - spleen_accuracy: 0.6691

2023-10-05 21:25:08.547519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


269/269 [==============================] - 463s 1s/step - loss: 3.4256 - bowel_loss: 0.7022 - extra_loss: 0.6416 - liver_loss: 0.6430 - kidney_loss: 0.5690 - spleen_loss: 0.8697 - bowel_accuracy: 0.5477 - extra_accuracy: 0.6705 - liver_accuracy: 0.8161 - kidney_accuracy: 0.8101 - spleen_accuracy: 0.6691 - val_loss: 3.1991 - val_bowel_loss: 0.6591 - val_extra_loss: 0.5675 - val_liver_loss: 0.3644 - val_kidney_loss: 0.6499 - val_spleen_loss: 0.9581 - val_bowel_accuracy: 0.5595 - val_extra_accuracy: 0.7773 - val_liver_accuracy: 0.9685 - val_kidney_accuracy: 0.8001 - val_spleen_accuracy: 0.5919
Epoch 2/5
269/269 [==============================] - 311s 1s/step - loss: 2.8336 - bowel_loss: 0.6158 - extra_loss: 0.5884 - liver_loss: 0.4953 - kidney_loss: 0.4247 - spleen_loss: 0.7095 - bowel_accuracy: 0.6685 - extra_accuracy: 0.7001 - liver_accuracy: 0.8338 - kidney_accuracy: 0.8233 - spleen_accuracy: 0.7200 - val_loss: 3.2255 - val_bowel_loss: 0.7080 - val_extra_loss: 0.5413 - val_liver_loss: 

In [11]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)))>

In [12]:
valid_df

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,...,spleen_high,any_injury,series_id,instance_number,injury_name,image_path,width,height,stratify,fold
184,10292,1,0,0,1,1,0,0,1,0,...,0,1,14945,20,Active_Extravasation,./Dataset/train_images/10292/14945/20.png,512,512,01100100100,3.0
185,10292,1,0,0,1,1,0,0,1,0,...,0,1,14945,21,Active_Extravasation,./Dataset/train_images/10292/14945/21.png,512,512,01100100100,3.0
186,10292,1,0,0,1,1,0,0,1,0,...,0,1,14945,22,Active_Extravasation,./Dataset/train_images/10292/14945/22.png,512,512,01100100100,3.0
187,10292,1,0,0,1,1,0,0,1,0,...,0,1,14945,23,Active_Extravasation,./Dataset/train_images/10292/14945/23.png,512,512,01100100100,3.0
188,10292,1,0,0,1,1,0,0,1,0,...,0,1,14945,24,Active_Extravasation,./Dataset/train_images/10292/14945/24.png,512,512,01100100100,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12988,9528,1,0,0,1,1,0,0,1,0,...,0,1,1989,217,Active_Extravasation,./Dataset/train_images/9528/1989/217.png,512,512,01100100100,3.0
12989,9528,1,0,0,1,1,0,0,1,0,...,0,1,1989,218,Active_Extravasation,./Dataset/train_images/9528/1989/218.png,512,512,01100100100,3.0
12990,9528,1,0,0,1,1,0,0,1,0,...,0,1,1989,219,Active_Extravasation,./Dataset/train_images/9528/1989/219.png,512,512,01100100100,3.0
12991,9528,1,0,0,1,1,0,0,1,0,...,0,1,1989,220,Active_Extravasation,./Dataset/train_images/9528/1989/220.png,512,512,01100100100,3.0
